**Machine Learning Basic Principles 2018 - Data Analysis Project Report**

# Genre Classification using SVM 

*Title should be concise and informative, describes the approach to solve the problem. Some good titles from previous years:*

*- Comparing extreme learning machines and naive bayes’ classifier in spam detection*

*- Using linear discriminant analysis in spam detection*

*Some not-so-good titles:*

*- Bayesian spam filtering with extras*

*- Two-component classifier for spam detection*

*- CS-E3210 Term Project, final report*




## Abstract

*Precise summary of the whole report, previews the contents and results. Must be a single paragraph between 100 and 200 words.*



## 1. Introduction

*Background, problem statement, motivation, many references, description of
contents. Introduces the reader to the topic and the broad context within which your
research/project fits*

*- What do you hope to learn from the project?*
*- What question is being addressed?*
*- Why is this task important? (motivation)*

*Keep it short (half to 1 page).*



## 2. Data analysis

*Briefly describe data (class distribution, dimensionality) and how will it affect
classification. Visualize the data. Don’t focus too much on the meaning of the features,
unless you want to.*

*- Include histograms showing class distribution.*



In [24]:
# Import libraries
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.svm import SVC
from sklearn import svm
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix

In [2]:
# Load the data and cleanup
dfx = pd.read_csv("train_data.csv", header = None)
dfy = pd.read_csv("train_labels.csv" , header= None)
dftest =  pd.read_csv("test_data.csv", header = None)
## Create Label List
Y_train=[]
for m in dfy.as_matrix().tolist():
    Y_train += m

/Users/rohitsaluja/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  import sys


In [4]:
# Normalise Test and Train data
np_scaled_train = preprocessing.quantile_transform(dfx)
X_train = pd.DataFrame(np_scaled_train)

np_scaled_test = preprocessing.quantile_transform(dftest)
X_test = pd.DataFrame(np_scaled_test)

In [4]:
# Analysis of the input data


In [5]:
# Correlation Matrix
corr_matrix = X_train.corr()
corr_matrix_absolute = X_train.corr().abs()

## 3. Methods and experiments

*- Explain your whole approach (you can include a block diagram showing the steps in your process).* 

*- What methods/algorithms, why were the methods chosen. *

*- What evaluation methodology (cross CV, etc.).*



In [6]:
# Feature Reduction using Upper Triangular Matrix from Corelation Matrix
upper_matrix = corr_matrix_absolute.where(np.triu(np.ones(corr_matrix_absolute.shape), k=1).astype(np.bool))
to_drop_index = [column for column in upper_matrix.columns if any(upper_matrix[column] > 0.90)]

X_train_reduced = X_train.drop(X_train.columns[to_drop_index], axis=1)
X_test_reduced = X_test.drop(X_test.columns[to_drop_index], axis=1)

In [7]:
# Best score for Support Vector Classification
parameter_candidates = [
  {'C': [1, 10, 100, 1000,10000], 'kernel': ['linear']},
  {'C': [1, 10, 100, 1000,10000], 'gamma': [0.1,0.01,0.001, 0.0001], 'kernel': ['rbf']},
]
clf = GridSearchCV(estimator=svm.SVC(), param_grid=parameter_candidates, n_jobs=-1)
clf.fit(X_train_reduced, Y_train) 
print('Best score for data1:', clf.best_score_) 
print('Best C:',clf.best_estimator_.C) 
print('Best Kernel:',clf.best_estimator_.kernel)
print('Best Gamma:',clf.best_estimator_.gamma)

KeyboardInterrupt: 

In [8]:
# Creating ML Model with SVM
model = SVC(kernel = 'rbf', C = 10,gamma=0.01, probability= True).fit(X_train_reduced, Y_train)

In [9]:
# Generating Predictions using Model for both Accuracy and Log-Loss
model_predictions_accuracy = model.predict(X_train_reduced)
model_predictions_log_loss = model.predict_proba(X_train_reduced)

In [20]:
Y_train_np = np.array(Y_train)

In [23]:
Y_train_np.shape

(4363,)

[1 1 1 ... 2 5 2]


## 4. Results

*Summarize the results of the experiments without discussing their implications.*

*- Include both performance measures (accuracy and LogLoss).*

*- How does it perform on kaggle compared to the train data.*

*- Include a confusion matrix.*



In [30]:
# Export Predictions to File

## Accuracy
np.savetxt("accuracy_solution.csv", 
           np.dstack((np.arange(1, model_predictions_accuracy.size+1),model_predictions_accuracy))[0],
           delimiter=',', comments="", fmt='%i,%i',
           header="Sample_id,Sample_label")

## Log_loss
sample_id_column = np.zeros((X_test_reduced.shape[0],1), dtype=int)
for i in range (X_test_reduced.shape[0]): 
    sample_id_column[i] = i+1
log_loss_solution = np.hstack((sample_id_column, model_predictions_log_loss))
np.savetxt("log_loss_solution.csv", log_loss_solution, 
           delimiter=',', comments="", fmt=','.join(['%i'] + ['%1.4f']*10),
           header="Sample_id,Class_1,Class_2,Class_3,Class_4,Class_5,Class_6,Class_7,Class_8,Class_9,Class_10")

In [15]:
log_loss_solution

array([[1.00000000e+00, 1.71488363e-01, 9.59349447e-02, ...,
        2.36661860e-03, 2.98687897e-02, 2.28177609e-02],
       [2.00000000e+00, 1.21822784e-01, 6.50170376e-02, ...,
        1.63510691e-02, 2.53553072e-01, 1.50261261e-02],
       [3.00000000e+00, 7.08710239e-01, 1.71647853e-02, ...,
        1.69019523e-01, 7.43674505e-04, 1.65138990e-02],
       ...,
       [6.54200000e+03, 7.34872245e-01, 7.72236360e-02, ...,
        3.93729047e-02, 2.83428058e-03, 3.00970439e-02],
       [6.54300000e+03, 1.07714478e-02, 3.39328879e-02, ...,
        2.17207393e-04, 1.28757705e-03, 2.94327376e-04],
       [6.54400000e+03, 5.64991559e-01, 4.18611627e-02, ...,
        7.51749602e-02, 2.30502493e-03, 6.69416657e-02]])

In [16]:
accuracy_solution

NameError: name 'accuracy_solution' is not defined

In [28]:
accuracy_score(Y_train_np,model_predictions_accuracy)


0.7648407059362824

In [33]:
#Confusion matrix ...
CM = confusion_matrix(Y_train_np,model_predictions_accuracy)
print(CM)


[[2063   64   18    8    1   18    0    3    3    0]
 [ 116  472   18    2    0    8    0    0    2    0]
 [  25   10  286    0    1    3    0    0    1    0]
 [  62   12    3  166    1    7    0    1    1    0]
 [ 122    6   11    6   57    7    0    2    3    0]
 [  86   10   13    8    3  138    0    1    0    1]
 [ 105   11    7    3    4    6    2    1    2    0]
 [ 105    1    1    3    1    3    0   81    0    0]
 [  11    3   13    0    1    3    0    0   61    0]
 [  62    1    3    3    1    3    0    2    0   11]]


In [32]:
CR = classification_report(Y_train_np,model_predictions_accuracy)
print(CR)

             precision    recall  f1-score   support

          1       0.75      0.95      0.84      2178
          2       0.80      0.76      0.78       618
          3       0.77      0.88      0.82       326
          4       0.83      0.66      0.73       253
          5       0.81      0.27      0.40       214
          6       0.70      0.53      0.61       260
          7       1.00      0.01      0.03       141
          8       0.89      0.42      0.57       195
          9       0.84      0.66      0.74        92
         10       0.92      0.13      0.22        86

avg / total       0.78      0.76      0.73      4363



## 5. Discussion/Conclusions

*Interpret and explain your results *

*- Discuss the relevance of the performance measures (accuracy and LogLoss) for
imbalanced multiclass datasets. *

*- How the results relate to the literature. *

*- Suggestions for future research/improvement. *

*- Did the study answer your questions? *



## 6. References

*List of all the references cited in the document*

## Appendix
*Any additional material needed to complete the report can be included here. For example, if you want to keep  additional source code, additional images or plots, mathematical derivations, etc. The content should be relevant to the report and should help explain or visualize something mentioned earlier. **You can remove the whole Appendix section if there is no need for it.** *